# Skyrmion Lattice with UppASD

2D skyrmion lattice example using the modern UppASD Python API.

In [ ]:
import importlib.util
import numpy as np
import matplotlib.pyplot as plt

if importlib.util.find_spec("uppasd") is None:
    print("Installing uppasd…")
    !pip install --pre --index-url https://test.pypi.org/simple/ --extra-index-url https://pypi.org/simple uppasd

from uppasd.core.system import SpinSystem
from uppasd.core.exchange import ExchangeShellTable, DMIShellTable
from uppasd.input.inputdata import ASDInput
from uppasd.run.simulator import ASDWorkspace, UppASDSimulator
from uppasd.core.results import ASDResults
from uppasd.viz.io import timeseries
from uppasd.viz.io import spin_snapshots, reshape_supercell
from uppasd.viz.heatmap import plot_heatmap
from uppasd.viz.trajectory import plot_2d_trajectory


## Define lattice

In [ ]:
# Set up a single unit cell with one atom
a = 1.0
cell = np.diag([a, a, a])

positions = np.array([[0.0 , 0.0, 0.0]])
natom = positions.shape[0]
species = np.ones(natom, dtype=int)

moments = np.zeros((natom, 3))
moments[:, 2] = 1.0

system = SpinSystem(cell, positions, species, moments)


## Interactions

In [ ]:
exchange = ExchangeShellTable()
dmi = DMIShellTable()

J = 1.0
D = 0.81650

for i in range(natom):
    x, y, _ = positions[i]
    for dx, dy, Dx, Dy in [(1,0,D,0),(-1,0,-D,0),(0,1,0,D),(0,-1,0,-D)]:
        jx = int((x+dx)/a) 
        jy = int((y+dy)/a)
        j = jx + jy
        exchange.add_bond(1, 1, 1, [dx,dy,0], J)
        dmi.add_bond(1, 1, 1, [dx,dy,0], [Dx,Dy,0])


## Input

In [ ]:
inp = ASDInput()
Nx=64
Ny=64
inp.block("system").set(ncell=(Nx,Ny,1), bc=("P","P","0"),do_prnstruct=2)
inp.block("initial").set(ip_mode="S", ip_nphase=(1 , '\n', 100, 0.0, 1e-15, 1.0), ip_hfield=(0.0, 0.0, -150.0), initmag=1)
inp.block("simulation").set(mode="S", nstep=5000, timestep=1e-15, 
                            damping=0.5, temperature=5.0,
                           hfield=(0.0, 0.0, -150.0))
inp.block("measurables").set(plotenergy=1, do_avrg="Y",
                             do_cumu="Y", do_prn_beff="Y",
                             skyno="T", do_skyno_den="Y",
                             do_skyno_cmass="Y", do_tottraj="Y")


## Run

In [ ]:
workdir = "skyrmion_lattice_run"
ws = ASDWorkspace(workdir, clean=True)
ws.prepare(system=system, inp=inp, exchange=exchange, dmi=dmi)

sim = UppASDSimulator(ws)
sim.initialize()
sim.run_init_phase()
sim.measure()
sim.finalize()


## Results

In [ ]:
results = ASDResults(workdir)
#results.summary()

print("Final energy:", results.final_energy)

## Magnetization heat map

In [ ]:
steps, spins = spin_snapshots(results, "restart")

ncell = (Nx, Ny, 1)
mz = reshape_supercell(spins[-1,:,2], ncell)[...,0]

plot_heatmap(mz, title="Final $m_z$ texture");

## Energy evolution

In [ ]:
step, e_xc = timeseries(results, "totenergy", column="exc")
_, e_dmi = timeseries(results, "totenergy", column="dm")
_, e_zm = timeseries(results, "totenergy", column="zeeman")


plt.figure()
plt.plot(step, e_xc, marker="o",label=r'E$_{xc}$')
plt.plot(step, e_dmi, marker="x",label=r'E$_{dm}$')
plt.plot(step, e_zm, marker="d",label=r'E$_{z}$')
plt.xlabel("Step")
plt.ylabel("Total energy")
plt.title("Energy vs time")
plt.legend()
plt.show()

## Center of mass

In [ ]:
cmass = results['cmass_skynum']
plot_2d_trajectory(cmass['rx'], cmass['ry'], title='Skyrmion center of mass trajectory');
